In [1]:
from skimage.io import imread_collection
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
import os
import itertools
from sklearn.utils import shuffle
from keras.models import load_model

Using TensorFlow backend.


ImportError: Could not find 'cudart64_90.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Download and install CUDA 9.0 from this URL: https://developer.nvidia.com/cuda-toolkit

# Training the model

In [61]:
leftCarTrainPath = os.getcwd()+"\\train-color\\yaw05\\"+"*.png"
rightCarTrainPath = os.getcwd()+"\\train-color\\yaw13\\"+"*.png"

leftCars = imread_collection(leftCarTrainPath)
rightCars = imread_collection(rightCarTrainPath)

In [62]:
datLs = list(zip(leftCars,itertools.repeat(0)))
datLs +=list(zip(rightCars,itertools.repeat(1)))

In [63]:
datLs = shuffle(datLs)

In [64]:
data = []
labels = []

for elm in datLs:
    data.append(elm[0])
    labels.append(elm[1])

The model architecture is from: https://keras.io/models/sequential/

In [4]:
x_train = np.array(data)
y_train = keras.utils.to_categorical(labels, num_classes=2)

model = Sequential()
# input: 160x240 images with 3 channels -> (160, 240, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(160, 240, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(x_train, y_train, batch_size=32, epochs=1)
#score = model.evaluate(x_test, y_test, batch_size=32)

NameError: name 'data' is not defined

In [68]:
model.save('carvana_vvg_1.h5')  

## Working with the saved model
Assumes you have the trained model stored in *carvana_vgg_1.h5*. This section evaluates the model on the test set. 

In [7]:
model = load_model('carvana_vvg_1.h5')

In [13]:
leftCarTestPath = os.getcwd()+"\\test-color\\yaw05\\"+"*.png"
rightCarTestPath = os.getcwd()+"\\test-color\\yaw13\\"+"*.png"

leftCarsTest = imread_collection(leftCarTestPath)
rightCarsTest = imread_collection(rightCarTestPath)

datLs = list(zip(leftCarsTest,itertools.repeat(0)))
datLs +=list(zip(rightCarsTest,itertools.repeat(1)))

data = []
labels = []

for elm in datLs:
    data.append(elm[0])
    labels.append(elm[1])
    
x_test = np.array(data)
y_test = keras.utils.to_categorical(labels, num_classes=2)

In [11]:
score = model.evaluate(x_test,  y_test, batch_size=32)

2628/2628 [==============================] - 82s 31ms/step


In [12]:
score

8.059047752682295